In [617]:
import pandas as pd
import json
import re
import numpy as np

In [618]:
# load the dataset
json_file = open('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_dataset.json', 'r')
#, encoding='utf-8'
data = json.load(json_file)

In [619]:
# Extract the variables of the first page
name = data[0]['name'].copy()
links = data[0]['link'].copy()
birth_year = data[0]['birth_year'].copy()
birth_place = data[0]['birth_place'].copy()
death_year = data[0]['death_year'].copy()
death_place = data[0]['death_place'].copy()

In [ ]:
# Get the full links of the second pages before creating the dataframe
full_links = []
for link in links:
    full = 'http://www.ifar.org/' + link
    full_links.append(full)

# Check that the variables was created successfully
full_links[:5], len(full_links)

In [623]:
# Create the first dataset containing all of page 1
page_one = pd.DataFrame(list(zip(name, full_links, birth_year, birth_place, death_year, death_place)),
                          columns = ['names', 'links', 'birth_year', 'birth_place', 'death_year', 'death_place'])

In [624]:
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,"<td class=""g_center_birth_year"">1552</td>","<td class=""g_center_birth_place"">Germany</td>","<td class=""g_center_death_year"">1615</td>","<td class=""g_center_death_place"">Czech Republi..."
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1898</td>","<td class=""g_center_birth_place"">Finland</td>","<td class=""g_center_death_year"">1976</td>","<td class=""g_center_death_place"">Finland</td>"
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1836</td>","<td class=""g_center_birth_place"">Italy</td>","<td class=""g_center_death_year"">1868</td>","<td class=""g_center_death_place"">Italy</td>"
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,"<td class=""g_center_birth_year"">1971</td>","<td class=""g_center_birth_place"">Algeria</td>","<td class=""g_center_death_year""></td>","<td class=""g_center_death_place""></td>"
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,"<td class=""g_center_birth_year"">1889</td>","<td class=""g_center_birth_place"">Spain</td>","<td class=""g_center_death_year"">1957</td>","<td class=""g_center_death_place"">Spain</td>"


In [625]:
# Clean all of the variables in the first page and add them again to the dataset
regex = r'^(\d{4})'
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

clean_byear = []
clean_bplace = []
clean_dyear = []
clean_dplace = []

for by in page_one['birth_year']:
    f = remove_html_tags(by)
    clean_byear.append(f)
    
for bp in page_one['birth_place']:
    q = remove_html_tags(bp)
    clean_bplace.append(q)
    
for dy in page_one['death_year']:
    e = remove_html_tags(dy)
    clean_dyear.append(e)
    
for dp in page_one['death_place']:
    w = remove_html_tags(dp)
    clean_dplace.append(w)

page_one['birth_year'] = pd.Series(clean_byear)
page_one['birth_place'] = pd.Series(clean_bplace)
page_one['death_year'] = pd.Series(clean_dyear)
page_one['death_place'] = pd.Series(clean_dplace)

In [626]:
# Make sure that the variables were cleaned correctly
page_one.tail()

,names,links,birth_year,birth_place,death_year,death_place
2932,"Zuccaro, Taddeo",http://www.ifar.org/artist_name.php?nameid=275...,1529,Italy,1566,Italy
2933,"Zuloaga, Ignacio",http://www.ifar.org/artist_name.php?nameid=273...,1870,Spain,1945,Spain
2934,"Zumthor, Peter",http://www.ifar.org/artist_name.php?nameid=269...,1943,Switzerland,,
2935,"Zuñiga, Francisco",http://www.ifar.org/artist_name.php?nameid=109...,1912,Costa Rica,1998,Mexico
2936,"Zurbarán, Francisco de",http://www.ifar.org/artist_name.php?nameid=110...,1598,Spain,1664,Spain


In [627]:
# Make a copy of the rest of the dataset
second_part = data[1:].copy()

In [628]:
len(second_part)

7926

In [630]:
# Clean all of the variables for the second and third pages
artist_one = []
box_info = []
artist_two = []
column_names = []
content = []

for s in second_part:
    for key, val in s.items():
        if key == 'artist_one':
            artist_one.append(val)
        elif key == 'box_info':
            box_info.append(val)
        elif key == 'artist_two':
            artist_two.append(val)
        elif key == 'column_names':
            column_names.append(val)
        elif key == 'content':
            content.append(val)
            
len(content), len(name)

(4989, 2937)

In [631]:
# Clean the box
box_new = []
for i in box_info:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    join_them = list(filter(None, tempora))
    box_new.append(join_them)

box_new[0], len(box_new)

(['Published Catalogues',
  'Hans von Aachen, 1552-1615.',
  'click to learn more',
  'Jacoby, Joachim W.',
  'Munich ; Berlin : Deutscher Kunstverlag, 2000.'],
 2937)

In [633]:
# Clean the new artist variable
artist_three = []
for i in artist_two:
    gen = str(i).strip('Artist: ')
    artist_three.append(gen)
    
artist_three[0], len(artist_three)

('Rilke-Westhoff, Clara', 4989)

In [634]:
# Clean the new columns
cols_news = []
for i in column_names:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    cols_news.append(tempora)
    
cols_news[1], len(cols_news)

(['Item Title:',
  'Author:',
  'ISBN:',
  'Imprint:',
  'Language:',
  'Size:',
  'Pages:',
  'Illustrations:',
  'Concordance:',
  'Bibliography:',
  'Index:',
  'Exhibition List:',
  'Chronology:',
  'Content Note:',
  'Individual Entries Contain:',
  'Public Note:'],
 4989)

In [903]:
new_content = []
for i in content:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    join_them = list(filter(None, tempora))
    new_content.append(tempora)
    
new_content[0], len(new_content)

(['Die Bildhauerin Clara Rilke-Westhoff, 1878-1954 : Leben und Werk (mit Oeuvre-Katalog).',
  'Sauer, Marina',
  '3920699726',
  'Bremen : Hauschild, 1986.',
  'German',
  '24 cm',
  '611',
  'numerous b&w',
  'No',
  'Yes',
  'Yes',
  'No',
  'No',
  'The chronological catalogue of the sculpture gives number, title, date, place, medium and dimensions, plus Rilke-Archiv information, exhibitions, references, and unpublished sources.  \n\nThe appendices include material on unidentified or undated works.',
  'Exhib. Hist; Bibliog.',
  'The introductory texts include a biography, an essay on the society of women artists at the turn of the twentieth century, and an analysis of the work.  There are commentaries on problems and the state of the research on the artist.  \n\nThere are lists of personal names, images, and abbreviations.'],
 4989)

In [904]:
double_list = list(zip(cols_news, new_content))
type(double_list[1])

tuple

In [905]:
momentary = []

for i in cols_news:
    for s in i:
        momentary.append(s)

x = np.array(momentary)

In [906]:
print(np.unique(x))

['Artist:' 'Author Type:' 'Author(s):' 'Author:' 'Bibliography:'
 'Catalogue:' 'Chronology:' 'Committee Member 1:' 'Committee Member 2:'
 'Committee Member 3:' 'Committee Name:' 'Concordance:'
 'Contact Information:' 'Content Note:' 'Continues Unpublished'
 'Date Begun:' 'Dissertation Author:' 'Dissertation:' 'Edition:'
 'Exhibition List:' 'Fee:' 'Format:' 'Fourth Author:'
 'Frequency of Review:' 'How Related:' 'How Revises:' 'ISBN:'
 'Illustrations:' 'Imprint:' 'Index:' 'Individual Entries Contain:'
 'Item Number:' 'Item Title:' 'Language(s):' 'Language:' 'Medium/Scope:'
 'Most Recent Edition:' 'Notes:' 'Original Publication Date:' 'Pages:'
 'Preferred Contact Method:' 'Primary Sponsor Type:' 'Primary Sponsor:'
 'Project Name:' 'Public Note:' 'Publication Date:' 'Publisher:'
 'Related Author(s):' 'Related Title:' 'Related to a Published'
 'Research by:' 'Reviewing:' 'Second Author:' 'Secondary Sponsor Type:'
 'Secondary Sponsor:' 'Size:' 'Third Author:' 'Title of Publication:'
 'Trans

In [907]:
pub_or_prep = []
for i in cols_news:
    if 'Item Title:' in i:
        pub_or_prep.append(1)
    else:
        pub_or_prep.append(2)
        
pub_or_prep[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 2, 1]

In [908]:
data_cols = ['Artist name', 'Item Title', 'Status (Published or in Preparation)', 'Author', 'ISBN', 'Imprint',
             'Language', 'Size', 'Pages', 'Illustrations', 'Concordance', 'Bibliography', 'Index',
             'Exhibition List', 'Chronology', 'Individual Entries Contain', 'Public Note']


In [909]:
len(third_page)

4989

In [913]:
dataset = pd.DataFrame(list(zip(artist_three, pub_or_prep, zip(cols_news, new_content))), columns = ['name', 'pub_or_prep', 'col_names_content'])
dataset.head(10)

,name,pub_or_prep,col_names_content
0,"Rilke-Westhoff, Clara",1,"([Item Title:, Author:, ISBN:, Imprint:, Langu..."
1,"Rho, Manlio",1,"([Item Title:, Author:, ISBN:, Imprint:, Langu..."
2,"Ribalta, Francisco",1,"([Item Title:, Author:, ISBN:, Imprint:, Langu..."
3,"Réquichot, Bernard",1,"([Item Title:, Author(s):, Imprint:, Language:..."
4,"Révoil, Pierre-Hen",1,"([Item Title:, Author:, Imprint:, Language:, S..."
5,"Riley, Bridge",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
6,"Riley, Bridge",1,"([Item Title:, Author(s):, ISBN:, Item Number:..."
7,"Riley, Bridge",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
8,"Tuculescu, Ion",2,"([Artist:, Medium/Scope:, Project Name:, Autho..."
9,"Trumbull, John",1,"([Item Title:, Author(s):, Imprint:, Language:..."


In [914]:
def gen_columns(series, name, new_list):
    for i in series:
        if name in i[0]:
            new_list.append(i[1][i[0].index(name)])
        else:
            new_list.append(np.nan)

In [ ]:
•	Artist name
•	Item title
•	Has this work been published or is still in preparation (this will be on the second page, once you click on the artist’s name, but before clicking on each catalogue)
•	Author name
•	ISBN
•	Imprint
•	Language
•	Size
•	Pages
•	Illustrations
•	Concordance
•	Bibliography
•	Index
•	Exhibition List
•	Chronology
•	Individuals entries contain
•	Public note


gen_columns(series, name, new_list)

In [932]:
artist = []
item_t = []
author = []
author_s = []
isbn = []
imprint = []
language = []
size = []
pages = []
illustrations = []
concordance = []
bibliography = []
index = []
exhibition_list = []
cronology = []
ind_entr_cont = []

gen_columns(series = dataset.col_names_content, name = 'Artist:', new_list = artist)
gen_columns(series = dataset.col_names_content, name = 'Item Title:', new_list = item_t)
gen_columns(series = dataset.col_names_content, name = 'Author:', new_list = author)
gen_columns(series = dataset.col_names_content, name = 'Author(s):', new_list = author_s)
gen_columns(series = dataset.col_names_content, name = 'ISBN:', new_list = isbn)
gen_columns(series = dataset.col_names_content, name = 'Imprint:', new_list = imprint)
gen_columns(series = dataset.col_names_content, name = 'Language:', new_list = language)
gen_columns(series = dataset.col_names_content, name = 'Size:', new_list = size)
gen_columns(series = dataset.col_names_content, name = 'Pages:', new_list = pages)
gen_columns(series = dataset.col_names_content, name = 'Illustrations:', new_list = illustrations)
gen_columns(series = dataset.col_names_content, name = 'Concordance:', new_list = concordance)
gen_columns(series = dataset.col_names_content, name = 'Bibliography:', new_list = bibliography)
gen_columns(series = dataset.col_names_content, name = 'Index:', new_list = index)
gen_columns(series = dataset.col_names_content, name = 'Exhibition List:', new_list = exhibition_list)
gen_columns(series = dataset.col_names_content, name = 'Chronology:', new_list = cronology)
gen_columns(series = dataset.col_names_content, name = 'Individual Entries Contain:', new_list = ind_entr_cont)

In [933]:
artist[:3], item_t[:3], author[:3], author_s[:3], isbn[:3], imprint[:3]

([nan, nan, nan],
 ['Die Bildhauerin Clara Rilke-Westhoff, 1878-1954 : Leben und Werk (mit Oeuvre-Katalog).',
  'Rho : catalogo generale.',
  'Ribalta y los ribaltescos : la evolución del estilo barroco en Valencia.'],
 ['Sauer, Marina', 'Caramel, Luciano', 'Kowal, David M.'],
 [nan, nan, nan],
 ['3920699726', '8843533339', '8450519811'],
 ['Bremen : Hauschild, 1986.',
  'Milan : Electa, 1990.',
  'Valencia : Diputación Provincial de Valencia [1985].'])

In [934]:
language[:3], size[:3], pages[:3], illustrations[:3], concordance[:3], bibliography[:3]

(['German', 'Italian', 'Spanish'],
 ['24 cm', '29 cm', '28 cm'],
 ['611', '316', '404'],
 ['numerous b&w', 'b&w, some color', 'b&w'],
 ['No', 'No', 'No'],
 ['Yes', 'Yes', 'Yes'])

In [935]:
index[:3], exhibition_list[:3], cronology[:3], ind_entr_cont[:3]

(['Yes', 'No', 'No'],
 ['No', 'Yes', 'No'],
 ['No', 'No', 'No'],
 ['Exhib. Hist; Bibliog.',
  'Provenance; Exhib. Hist; Bibliog.; Comments',
  'Comments'])

In [936]:
public_note = []
track = []
for i in dataset.col_names_content:
    if 'Public Note:' in i[0]:
        try:
            public_note.append(i[1][i[0].index('Public Note:')])
        except IndexError:
            public_note.append('check_needed')
    else:
        public_note.append(np.nan)
        
len(public_note)

4989

In [937]:
variables = [
             artist, item_t, author, author_s, isbn, imprint, language, size, pages, illustrations,
             concordance, bibliography, index, exhibition_list, cronology, ind_entr_cont, public_note
]

In [938]:
df_final = dataset.copy()

In [939]:
df_final = df_final.drop(columns='col_names_content')
df_final.head()

,name,pub_or_prep
0,"Rilke-Westhoff, Clara",1
1,"Rho, Manlio",1
2,"Ribalta, Francisco",1
3,"Réquichot, Bernard",1
4,"Révoil, Pierre-Hen",1


In [940]:
df_final['artist'] = pd.Series(artist)
df_final['item_t'] = pd.Series(item_t)
df_final['author'] = pd.Series(author)
df_final['author_s'] = pd.Series(author_s)
df_final['isbn'] = pd.Series(isbn)
df_final['imprint'] = pd.Series(imprint)
df_final['language'] = pd.Series(language)
df_final['size'] = pd.Series(size)
df_final['pages'] = pd.Series(pages)
df_final['illustrations'] = pd.Series(illustrations)
df_final['concordance'] = pd.Series(concordance)
df_final['bibliography'] = pd.Series(bibliography)
df_final['index'] = pd.Series(index)
df_final['exhibition_list'] = pd.Series(exhibition_list)
df_final['cronology'] = pd.Series(cronology)
df_final['ind_entr_cont'] = pd.Series(ind_entr_cont)
df_final['public_note'] = pd.Series(public_note)

df_final.head()

,name,pub_or_prep,artist,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Rilke-Westhoff, Clara",1,NaN,"Die Bildhauerin Clara Rilke-Westhoff, 1878-195...","Sauer, Marina",NaN,3920699726,"Bremen : Hauschild, 1986.",German,24 cm,611,numerous b&w,No,Yes,Yes,No,No,Exhib. Hist; Bibliog.,"The introductory texts include a biography, an..."
1,"Rho, Manlio",1,NaN,Rho : catalogo generale.,"Caramel, Luciano",NaN,8843533339,"Milan : Electa, 1990.",Italian,29 cm,316,"b&w, some color",No,Yes,No,Yes,No,Provenance; Exhib. Hist; Bibliog.; Comments,"After the introduction, there is a catalogue r..."
2,"Ribalta, Francisco",1,NaN,Ribalta y los ribaltescos : la evolución del e...,"Kowal, David M.",NaN,8450519811,Valencia : Diputación Provincial de Valencia [...,Spanish,28 cm,404,b&w,No,Yes,No,No,No,Comments,This is related to the author's 1981 doctoral ...
3,"Réquichot, Bernard",1,NaN,Bernard Réquichot.,NaN,"Barthes, Roland ...",NaN,"Brussels : La Connaissance, 1973",French,31 cm,301,"numerous b&w thumbnails, plates",No,Yes,No,Yes,No,NaN,The catalogue raisonné follows the introductor...
4,"Révoil, Pierre-Hen",1,NaN,Fleury Richard et Pierre Révoil : la peinture ...,"Chaudonneret, Marie-Claude",NaN,NaN,"Paris : Arthena, 1980.",French,29 cm,217,"b&w, some color",No,Yes,Yes,No,No,NaN,NaN


In [954]:
df_final.to_csv('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_data.csv', index=False, encoding='utf-8')

In [955]:
df = pd.read_csv('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_data.csv')

In [958]:
df.head()

,name,pub_or_prep,artist,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Rilke-Westhoff, Clara",1,NaN,"Die Bildhauerin Clara Rilke-Westhoff, 1878-195...","Sauer, Marina",NaN,3920699726,"Bremen : Hauschild, 1986.",German,24 cm,611,numerous b&w,No,Yes,Yes,No,No,Exhib. Hist; Bibliog.,"The introductory texts include a biography, an..."
1,"Rho, Manlio",1,NaN,Rho : catalogo generale.,"Caramel, Luciano",NaN,8843533339,"Milan : Electa, 1990.",Italian,29 cm,316,"b&w, some color",No,Yes,No,Yes,No,Provenance; Exhib. Hist; Bibliog.; Comments,"After the introduction, there is a catalogue r..."
2,"Ribalta, Francisco",1,NaN,Ribalta y los ribaltescos : la evolución del e...,"Kowal, David M.",NaN,8450519811,Valencia : Diputación Provincial de Valencia [...,Spanish,28 cm,404,b&w,No,Yes,No,No,No,Comments,This is related to the author's 1981 doctoral ...
3,"Réquichot, Bernard",1,NaN,Bernard Réquichot.,NaN,"Barthes, Roland ...",NaN,"Brussels : La Connaissance, 1973",French,31 cm,301,"numerous b&w thumbnails, plates",No,Yes,No,Yes,No,NaN,The catalogue raisonné follows the introductor...
4,"Révoil, Pierre-Hen",1,NaN,Fleury Richard et Pierre Révoil : la peinture ...,"Chaudonneret, Marie-Claude",NaN,NaN,"Paris : Arthena, 1980.",French,29 cm,217,"b&w, some color",No,Yes,Yes,No,No,NaN,NaN
